In [29]:
from prondict import prondict
from path import my_path
import os
from lab3_tools import *
from lab3_proto import *

from lab1_tools import *
from lab1_proto import *

from lab2_tools import *
from lab2_proto import *

4.2 Forced Alignment

In [30]:
 
phoneHMMs = np.load('lab2_models_all.npz',allow_pickle=True)['phoneHMMs'].item()
phones = sorted(phoneHMMs.keys())
nstates = {phone: phoneHMMs[phone]['means'].shape[0] for phone in phones}
stateList = [ph + '_' + str(id) for ph in phones for id in range(nstates[ph])]
# phoneHMMs contains Hmms for different phones 
print(phoneHMMs.keys())
print(phoneHMMs["f"].keys())
print(stateList)

dict_keys(['f', 'k', 'n', 'r', 's', 't', 'v', 'w', 'z', 'ah', 'ao', 'ay', 'eh', 'ey', 'ih', 'iy', 'ow', 'sp', 'th', 'uw', 'sil'])
dict_keys(['name', 'startprob', 'transmat', 'means', 'covars'])
['ah_0', 'ah_1', 'ah_2', 'ao_0', 'ao_1', 'ao_2', 'ay_0', 'ay_1', 'ay_2', 'eh_0', 'eh_1', 'eh_2', 'ey_0', 'ey_1', 'ey_2', 'f_0', 'f_1', 'f_2', 'ih_0', 'ih_1', 'ih_2', 'iy_0', 'iy_1', 'iy_2', 'k_0', 'k_1', 'k_2', 'n_0', 'n_1', 'n_2', 'ow_0', 'ow_1', 'ow_2', 'r_0', 'r_1', 'r_2', 's_0', 's_1', 's_2', 'sil_0', 'sil_1', 'sil_2', 'sp_0', 't_0', 't_1', 't_2', 'th_0', 'th_1', 'th_2', 'uw_0', 'uw_1', 'uw_2', 'v_0', 'v_1', 'v_2', 'w_0', 'w_1', 'w_2', 'z_0', 'z_1', 'z_2']


In [31]:
filename = my_path + 'tidigits\\disc_4.1.1\\tidigits\\train\\man\\nw\\z43a.wav'
samples, samplingrate = loadAudio(filename)
lmfcc = mfcc(samples)

#Extract the words from 
wordTrans = list(path2info(filename)[2])
#Extract the phones from the words
phoneTrans = words2phones(wordTrans, prondict)

print(wordTrans)
print(phoneTrans)

['z', '4', '3']
['sil', 'z', 'iy', 'r', 'ow', 'sp', 'f', 'ao', 'r', 'sp', 'th', 'r', 'iy', 'sp', 'sil']


In [32]:
#create a combined model for the utterance
utteranceHMM = concatHMMs(phoneHMMs, phoneTrans)

In [33]:
#map the states in utteranceHMM into the unique state names in stateList
stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
print(stateTrans)

['sil_0', 'sil_1', 'sil_2', 'z_0', 'z_1', 'z_2', 'iy_0', 'iy_1', 'iy_2', 'r_0', 'r_1', 'r_2', 'ow_0', 'ow_1', 'ow_2', 'sp_0', 'f_0', 'f_1', 'f_2', 'ao_0', 'ao_1', 'ao_2', 'r_0', 'r_1', 'r_2', 'sp_0', 'th_0', 'th_1', 'th_2', 'r_0', 'r_1', 'r_2', 'iy_0', 'iy_1', 'iy_2', 'sp_0', 'sil_0', 'sil_1', 'sil_2']


In [34]:
viterbiStateTrans = forcedAlignment(lmfcc,phoneHMMs,phoneTrans,stateTrans)
frames2trans(viterbiStateTrans, outfilename='z43a.lab')

c:\Users\cmp23\Skrivbord\DT2119-Speech-and-Speaker-Recognition_labs\Lab 3\Hasan\lab3_proto.py:52: RuntimeWarning: divide by zero encountered in log
  _, viterb_path = viterbi(obsloglik, np.log(utteranceHMM['startprob']), np.log(utteranceHMM['transmat'][:-1, :-1]))


'0 0.02 sil_0\n0.02 0.20000000000000004 sil_1\n0.20000000000000004 0.21000000000000005 sil_2\n0.21000000000000005 0.25000000000000006 z_0\n0.25000000000000006 0.26000000000000006 z_1\n0.26000000000000006 0.37000000000000016 z_2\n0.37000000000000016 0.45000000000000023 iy_0\n0.45000000000000023 0.46000000000000024 iy_1\n0.46000000000000024 0.47000000000000025 iy_2\n0.47000000000000025 0.5700000000000003 r_0\n0.5700000000000003 0.5800000000000003 r_1\n0.5800000000000003 0.5900000000000003 r_2\n0.5900000000000003 0.6000000000000003 ow_0\n0.6000000000000003 0.6100000000000003 ow_1\n0.6100000000000003 0.7000000000000004 ow_2\n0.7000000000000004 0.7100000000000004 f_0\n0.7100000000000004 0.8200000000000005 f_1\n0.8200000000000005 0.8300000000000005 f_2\n0.8300000000000005 0.8400000000000005 ao_0\n0.8400000000000005 0.9800000000000006 ao_1\n0.9800000000000006 1.0900000000000007 ao_2\n1.0900000000000007 1.1200000000000008 r_0\n1.1200000000000008 1.1300000000000008 r_1\n1.1300000000000008 1.140

In [35]:
'''
traindata = []
for root, dirs, files in os.walk(my_path +  'tidigits\\disc_4.1.1\\tidigits\\train'):
 for file in files:
  if file.endswith('.wav'):
    filename = os.path.join(root, file)
    samples, samplingrate = loadAudio(filename)
    lmfcc_uttarnce = mfcc(samples)
    mspec_uttarnce = mspec(samples)
    

    wordTrans = list(path2info(filename)[2])
    
    phoneTrans = words2phones(wordTrans, prondict)
    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
    viterbiStateTrans = forcedAlignment(lmfcc_uttarnce,phoneHMMs,phoneTrans,stateTrans)
    
    targets = []
    for state in viterbiStateTrans:
      targets.append(stateList.index(state))
  
    traindata.append({'filename': filename, 'lmfcc': lmfcc_uttarnce,'mspec': mspec_uttarnce, 'targets': targets})

'''

"\ntraindata = []\nfor root, dirs, files in os.walk(my_path +  'tidigits\\disc_4.1.1\\tidigits\\train'):\n for file in files:\n  if file.endswith('.wav'):\n    filename = os.path.join(root, file)\n    samples, samplingrate = loadAudio(filename)\n    lmfcc_uttarnce = mfcc(samples)\n    mspec_uttarnce = mspec(samples)\n    \n\n    wordTrans = list(path2info(filename)[2])\n    \n    phoneTrans = words2phones(wordTrans, prondict)\n    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]\n    viterbiStateTrans = forcedAlignment(lmfcc_uttarnce,phoneHMMs,phoneTrans,stateTrans)\n    \n    targets = []\n    for state in viterbiStateTrans:\n      targets.append(stateList.index(state))\n  \n    traindata.append({'filename': filename, 'lmfcc': lmfcc_uttarnce,'mspec': mspec_uttarnce, 'targets': targets})\n\n"

In [36]:
#np.savez('traindata.npz', traindata=traindata)

In [37]:
'''
testdata = []
for root, dirs, files in os.walk(my_path +  'tidigits\\disc_4.2.1\\tidigits\\test'):
 for file in files:  
  if file.endswith('.wav'):
    filename = os.path.join(root, file)
    samples, samplingrate = loadAudio(filename)
    lmfcc_uttarnce = mfcc(samples)
    mspec_uttarnce = mspec(samples)
    

    wordTrans = list(path2info(filename)[2])
    
    phoneTrans = words2phones(wordTrans, prondict)
    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]
    viterbiStateTrans = forcedAlignment(lmfcc_uttarnce,phoneHMMs,phoneTrans,stateTrans)
    

    targets = []
    for state in viterbiStateTrans:
      targets.append(stateList.index(state))
  
    testdata.append({'filename': filename, 'lmfcc': lmfcc_uttarnce,'mspec': mspec_uttarnce, 'targets': targets})

'''



"\ntestdata = []\nfor root, dirs, files in os.walk(my_path +  'tidigits\\disc_4.2.1\\tidigits\\test'):\n for file in files:  \n  if file.endswith('.wav'):\n    filename = os.path.join(root, file)\n    samples, samplingrate = loadAudio(filename)\n    lmfcc_uttarnce = mfcc(samples)\n    mspec_uttarnce = mspec(samples)\n    \n\n    wordTrans = list(path2info(filename)[2])\n    \n    phoneTrans = words2phones(wordTrans, prondict)\n    stateTrans = [phone + '_' + str(stateid) for phone in phoneTrans for stateid in range(nstates[phone])]\n    viterbiStateTrans = forcedAlignment(lmfcc_uttarnce,phoneHMMs,phoneTrans,stateTrans)\n    \n\n    targets = []\n    for state in viterbiStateTrans:\n      targets.append(stateList.index(state))\n  \n    testdata.append({'filename': filename, 'lmfcc': lmfcc_uttarnce,'mspec': mspec_uttarnce, 'targets': targets})\n\n"

In [38]:
#np.savez('testdata.npz', testdata=testdata)

In [39]:
traindata = np.load('traindata.npz',allow_pickle=True)['traindata']


In [40]:
data_train = []
data_validation =[]

#6 men and 6 women to include in the validation set
#It will be 10 % of the entire data
id_validation = ['ae','aj','al','aw','bd','cb','ac','ag', 'ai', 'an','bh','bi']

counter_validation = 0
counter_train = 0
counter_all = 0

validation_c_man = 0
validation_c_women = 0

train_c_man = 0
train_c_women = 0

for i in range((len(traindata))):
  file = traindata[i]['filename']
  id = path2info(file)[1]
  gender = path2info(file)[0]

  N = traindata[i]['lmfcc'].shape[0]
  counter_all += N

  
  if id in id_validation:
   counter_validation  += N
   data_validation.append(traindata[i])
   if gender=='man': 
    validation_c_man += N
   if gender=='woman':
    validation_c_women +=N
        
  else:
   data_train.append(traindata[i])  
   counter_train +=N 
   if gender=='man': 
    train_c_man += N
   if gender=='woman':
    train_c_women +=N   
   

print("Number of data: " + str(counter_all))

print("The percentage of validation_data: " + str(100*np.round(counter_validation/counter_all,2)))
print("The percentage of training_data:: " + str(100*np.round(counter_train/counter_all,2)))
print()
print("The persenatge of men in validation: "   + str(100*np.round(validation_c_man/counter_validation,2)))
print("The persenatge of women in validation: " + str(100*np.round(validation_c_women/counter_validation,2)))
print()
print("The percentage of men in the training_data: " + str(100*np.round(train_c_man/counter_train,2)))
print("The percentage of women in the training_data: " + str(100*np.round(train_c_women/counter_train,2)))

Number of data: 1507392
The percentage of validation_data: 10.0
The percentage of training_data:: 90.0

The persenatge of men in validation: 50.0
The persenatge of women in validation: 50.0

The percentage of men in the training_data: 48.0
The percentage of women in the training_data: 52.0


In [41]:
def get_indicies(index , last_index):
  left_index = index
  right_index = index
  t = [index]
  for i in range(3):
    
    left_index -=1
    t.insert(0,left_index)

    if(right_index==last_index):
       right_index =0
       t.append(right_index)
    else:
        right_index +=1 
        t.append(right_index)

  for i in range(len(t)):
    if(t[i]<0):
      t[i] = -1 * t[i]

  return t    





def Dynamic_Features(x):
  
  g = np.zeros((x.shape[0], 7* x.shape[1]))
  

  for i in range(x.shape[0]):
     list = get_indicies(i,x.shape[0]-1)
     temp = x[list ,:]
     g[i] = temp.flatten()

  return np.array(g)   

